## Projeto de Consulta Juridica

Configuração de navegador

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
#from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

#servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome()

Abrir a página index (entrar no site da busca jurídica)

In [15]:
import os
caminho = os.getcwd()
arquivo = caminho + r"\templates\index.html"

Base de dados

In [16]:
import pandas as pd

tabela = pd.read_excel(r'Base de Dados\Processos.xlsx')
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


Estrutura de Consulta Jurídica

In [17]:
from operator import index
from selenium.webdriver import ActionChains
import time 
from cgitb import text

for linha in tabela.index:
  # para cada processo (linha da tabela)
  navegador.get(arquivo)

  #abrir lista de cidades
  botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
  ActionChains(navegador).move_to_element(botao).perform()

  cidade = tabela.loc[linha, 'Cidade']
  print(cidade)

  # Selecionando o Rio de Janeiro
  navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

  # mudar para a nova aba
  aba_original = navegador.window_handles[0]
  indice = 1 + linha
  nova_aba = navegador.window_handles[indice]

  navegador.switch_to.window(nova_aba)
  # preencher o formulário com os dados de busca
  navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, "Nome"])
  navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, "Advogado"])
  navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, "Processo"])

  # clicar em pesquisar
  navegador.find_element(By.CLASS_NAME, 'registerbtn').click()
  
  # confirmar a pesquisa
  alerta = navegador.switch_to.alert
  alerta.accept()

  # esperar o resultado da pesquisa e agir de acordo com o resultado
  
  while True:
    try:
      alerta = navegador.switch_to.alert
      break
    except:
      time.sleep(1)
  texto_alerta = alerta.text

  if 'Processo encontrado com sucesso' in texto_alerta:
    alerta.accept()
    tabela.loc[linha, "Status"] = "Encontrado"
  else:
    tabela.loc[linha, "Status"] = "Não encontrado"
    alerta.accept()

Distrito Federal
Rio de Janeiro
Rio de Janeiro
São Paulo


Encerramento de navegador e atualização do Arquivo Processado 

In [18]:
navegador.quit()
display(tabela)

tabela.to_excel(r"Base de dados\Processo Atualizado.xlsx")

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Não encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Não encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Não encontrado
3,Carol,Amanda,PC5197,São Paulo,Encontrado
